# Training the YOLOv5 Model


In [ ]:
import os
from yolov5 import train

def train_yolo_model(train_folder, epochs=5):
    # Define the arguments as a dictionary
    opt = {
        'data': os.path.join(train_folder, 'data.yaml'),  # Path to the data configuration file
        'weights': 'yolov5s.pt',  # Pretrained weights
        'epochs': epochs,  # Number of epochs
        'project': 'runs/train',  # Save results to 'runs/train' directory
        'name': 'exp',  # Name of the experiment
    }
    
    # Call the train function with the arguments
    train.main(opt)

if __name__ == "__main__":
    # Define paths
    train_folder = 'depth_images/training'
    
    # Train the model
    train_yolo_model(train_folder)


# Testing the YOLOv5 Model


In [ ]:
from yolov5 import detect

def test_yolo_model(test_folder, model_path):
    # Define the arguments as a dictionary
    opt = {
        'weights': model_path,  # Path to the trained model
        'source': test_folder,  # Directory of test images
        'project': 'results',  # Save results to 'results' directory
        'name': 'exp',  # Name of the experiment
    }
    
    # Call the detect function with the arguments
    detect.main(opt)

if __name__ == "__main__":
    # Define paths
    test_folder = 'depth_images/testing'
    
    # Test the model
    test_yolo_model(test_folder, 'runs/train/exp/weights/best.pt')


In [ ]:
import os
from pathlib import Path
from yolov5 import train as train_model
from yolov5 import detect as detect_model

def train_yolo_model(train_folder, epochs=5):
    # Train the model using the train.py script from YOLOv5
    train_model.run(
        data=os.path.join(train_folder, 'data.yaml'),  # Path to the data configuration file
        weights='yolov5s.pt',  # Pretrained weights
        epochs=epochs,  # Number of epochs
        project='runs/train',  # Save results to 'runs/train' directory
        name='exp',  # Name of the experiment
    )

def test_yolo_model(test_folder, model_path):
    # Perform inference using the detect.py script from YOLOv5
    detect_model.run(
        weights=model_path,  # Path to the trained model
        source=test_folder,  # Directory of test images
        project='results',  # Save results to 'results' directory
        name='exp',  # Name of the experiment
    )

if __name__ == "__main__":
    # Define paths
    train_folder = 'depth_images/training'
    test_folder = 'depth_images/testing'
    
    # Train the model
    train_yolo_model(train_folder)
    
    # Test the model
    test_yolo_model(test_folder, 'runs/train/exp/weights/best.pt')


In [15]:
import os
import glob
import cv2

# Map class names to indices
class_map = {
    'Car': 0,
    'Van': 1,
    'Truck': 2,
    'Pedestrian': 3,
    'Person_sitting': 4,
    'Cyclist': 5,
    'Tram': 6,
    'Misc': 7,
    'DontCare': 8
}

# Define the path to the KITTI labels directory
kitti_labels_dir = './labels'

# Define the path to the KITTI images directory
kitti_images_dir = './depth_images/training'

# Define the output directory for YOLO format labels
yolo_labels_dir = './yolo_labels'
os.makedirs(yolo_labels_dir, exist_ok=True)

# Loop through all label files
for label_file in glob.glob(os.path.join(kitti_labels_dir, '*.txt')):
    # Open the label file
    with open(label_file, 'r') as f:
        lines = f.readlines()

    # Get the corresponding image file
    directory, filename = os.path.split(label_file)
    name, ext = os.path.splitext(filename)

    # Modify the file name by appending '_depth' and changing the extension to .png
    new_filename = f"{name}_depth.png"
    image_file = os.path.join(kitti_images_dir, new_filename)

    # Read the image dimensions
    img = cv2.imread(image_file)
    height, width, _ = img.shape

    # Open the output YOLO format file
    yolo_label_file = os.path.join(yolo_labels_dir, os.path.basename(label_file))
    with open(yolo_label_file, 'w') as f:
        # Loop through each object in the label file
        for line in lines:
            parts = line.strip().split(' ')
            obj_class = parts[0]
            if obj_class not in class_map:
                continue

            # Convert KITTI format to YOLO format
            class_idx = class_map[obj_class]
            bbox_xmin = float(parts[4])
            bbox_ymin = float(parts[5])
            bbox_xmax = float(parts[6])
            bbox_ymax = float(parts[7])
            x_center = (bbox_xmin + bbox_xmax) / 2 / width
            y_center = (bbox_ymin + bbox_ymax) / 2 / height
            bbox_width = (bbox_xmax - bbox_xmin) / width
            bbox_height = (bbox_ymax - bbox_ymin) / height

            # Write the YOLO format line
            f.write(f'{class_idx} {x_center} {y_center} {bbox_width} {bbox_height}\n')